In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from functions import base_functions
from tqdm import tqdm
import math
from sklearn.linear_model import LinearRegression
from SymbolicNet import SymbolicNet
import pandas as pd
import numpy as np

In [2]:
x = torch.tensor(pd.read_csv("x_and_u.csv", header=None).to_numpy(dtype=np.float32))
y = torch.tensor(pd.read_csv("x_dot.csv", header=None).to_numpy(dtype=np.float32))

In [4]:
features = base_functions
x_temp = []

# Engineer features
x_engineered = torch.stack([feature.torch(x) for feature in features], dim=1)
 
#x = x.view(-1, 1)

In [5]:
print(len(x_engineered))

49990


In [7]:
print(x_engineered.shape)

torch.Size([49990, 3, 13])


In [2]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()

In [5]:
x.shape

torch.Size([49990, 13])

In [6]:
coefs = []
for i in range(y.shape[1]):
    y_coefs = []
    for j in range(x_engineered.shape[2]):
        linear_model.fit(x_engineered[:, :, j], y[:, i])
        y_coefs.append(linear_model.coef_)
    coefs.append(y_coefs)

In [9]:
coefs

[[array([ 0.0173217 ,  0.07428496, -0.04577153], dtype=float32),
  array([ 0.10448315, -0.07413026, -0.0295673 ], dtype=float32),
  array([-0.16026908, -0.18149248,  0.1630214 ], dtype=float32),
  array([ 0.07454333,  0.17785338, -0.13899475], dtype=float32),
  array([ 0.16653283,  0.64512575, -0.54418916], dtype=float32),
  array([ 0.23605353,  0.9538725 , -0.99171895], dtype=float32),
  array([-0.15576021,  0.07620607,  0.09433444], dtype=float32),
  array([-0.16118358, -0.05044907,  0.10193407], dtype=float32),
  array([-5.9782948e-31,  1.6543612e-24, -2.1815537e-17], dtype=float32),
  array([-0.0626258 , -0.47194093,  0.11676586], dtype=float32),
  array([-7.1449406e-41,  0.0000000e+00, -1.3801471e-22], dtype=float32),
  array([1.2188494e-41, 0.0000000e+00, 2.7224975e-23], dtype=float32),
  array([ 0.13153484,  0.0148347 , -0.07950458], dtype=float32)],
 [array([-0.00491302, -0.00734284,  0.00734222], dtype=float32),
  array([ 0.04124124, -0.05128219, -0.00822325], dtype=float32),


In [34]:
thetas = linear_model.coef_

In [8]:
# Create list where features are sorted by importance
features = [[feature, importance] for feature, importance in zip(features, thetas)]
features.sort(key=lambda x: abs(x[1]), reverse=True)

In [9]:
from functions import Constant

optimal_features = [feature for feature in features if feature[1] > 0.05 or type(feature[0]) == type(Constant())]

In [10]:
functions = [feature[0] for feature in optimal_features]

print(f"{[func.sympy() for func in functions]} are to most important features.")

['x'] are to most important features.


In [11]:
model = SymbolicNet(1, len(functions), 1, base_functions)

In [12]:
x = x[:, 1].view(-1, 1)
y = y[:, 1].view(-1, 1)

In [14]:
model.train(True)
model.train_n_epochs(x, y, epochs=50, lr=0.01)

100%|██████████| 49990/49990 [00:05<00:00, 9994.42it/s] 


Epoch: 0 Loss: 0.008883344940841198


100%|██████████| 49990/49990 [00:04<00:00, 10298.42it/s]


In [15]:
weights = model.get_weights()

In [16]:
print(weights)

Parameter containing:
tensor([[ 0.2880, -0.0398]], requires_grad=True)


In [17]:
for w, f in zip(weights[0][0], model.functions):
    print(w.item(), f.sympy())
    

TypeError: iteration over a 0-d tensor